In [ ]:
    #catalogue
#1) global_alignment
#2) traceback
#3) match_fn
#4) global_alignment_matrix
#5) count_kmers1
#6) decapeptide_filter_query
#7) decapeptide_filter
#8) ktuple_filter
#9) import_df_by_part2
#10) import_df_by_part1
#11) import_df_by_part
#12) substring_occurrences
#13) search_polyA
#14) generate_impure_pA
#15) find_tr_ORF_stats
#16) which_frame
#17) which_frame_by_coord
#18) flnc_tr_seq_stats
#19) prot_properties
#20) rev_comp
#21) translate
#22) read_blast
#23) calc_overlaps
#24) overlaps_to_plot
#25) gtf_to_plot
#26) parse_fasta
#27) write_fasta_from_fasta_df
#28) write_fasta_from_MREdf
#29) write_fasta_from_overlap_df1
#30) uniques_only
#31) uniques_only_list
#32) write_list
#33) read_list
#34) find_fltc_ORFs
#35) find_ORFs_old
#36) list_duplicates
#37) fasta_subseqs
#38) calc_nucl_freq
#39) calc_aa_freq
#40) count_aas
#41) DNA_onehot
#42) count_kmers
#43) find_overrepresented_aas
#44) mann_whitney
#45) t_test
#46) extract_nested_values
#47) PCA_feature_importance
#48) convert_tr_ids
#49) calc_GC

In [ ]:
#MSA functions:
#1) get distance matrix of pairwise global alignments, count differences between two aligned strings 
#def count_diffs(a1, a2):
#when a1[i]!=a2[i]
# diff_score +=1
#def alignment_diffs(s1, s2, gap, match_fn):
#call global_alignment, then count_diffs
#def guidetree_matrix(sequences, gap, match_fn):
#initialise matrix
#for i in range(len(sequences[:-1])):
# for j in range(len(sequences[1:])):
#matrix[i,j] and matrix[j,i] == alignment_diffs(sequences[i],sequences[j])

#2) Guide tree traversal:
#def find_nearest(mat):
# find smallest distance in the matrix
# set min_val = np.max(mat)+1
# for row[0:-1], from col starting in row+1:
# if mat[row, col] <min_val, set min_row and min_col and min_val
# return values of min_row, min_col, and min_val
#def UPGMA_step(mat):
#use find_nearest() to merge elements i1 and i2 (nearest elems)
#no.copy(mat)
#set row i1 to average of i1 and i2 in origincal matrix
#same for column i1
#set [i1,i1] =0
#delete row and column i2 with np.delete(new_mat, i2, axis=0) (and axis=1)
#return i1, i2, and new matrix

#3) compute consensus
#def column_consensus(mat, col):
#create a list with residue in position col of all sequences in msa
# compute mode of this list (scipy.stats import mode)
# this function returns most common residue and count
#if count[0]<one half of sequences or mode[0] is negative, return "X"
#else return mode[0]
#def create_consensus(msa):
#start with an empty string and add consensuss for each column

#4) merge MSA
#align consensi
#pad with gaps in consensus alignment
#join two MSAs (add lists) and recompute consensus
#def pad_msa(msa, alignment):
#create new msa=[""]*len(msa) and current_pos = 0
# for each index in alignment:
#if "-", add "-" to each string in new_msa
#else, add character in current_pos of each sequence
#increment current_pos
#def combine_msa(old_list, i1, i2, new):
#create empty list
#for eac index in old_list, if it is i1 append the new element
#if it is !=i2, append that element from old list (discards i2)

#5) follow the guidetree
#def clustal_step( matrix, msas, consensus, names, gap, match_fn):
#i1,i2,new_mat = UPGMA_step(matrix)
#a1,a2,score = global_alignment(consensus[i1], consensus[i2], gap, match_fn)
#new_msa= pad_msa(msas[i1],a1)+pad_msa(msas[i2],a2)
#new_msas=combine_elements(msas, i1,i2,new_msa)
#new_consensus=combine_elements(consensus,i1,i2,create_consensus(new_msa))
#new_names = combine_elements(names,i1,i2,names[i1]+names[i2])
#return new_mat,new_msas, new_consensus, new_names
#
#def do_clustal(sequences, names, gap, match_fn):
#get matrix from guidetree_matrix(sequences,gap,match_fn)
#while more than one element (in msas, names of consensus):
#mat,msas,consensus,name_groups = clustal_step(...)
#return msas[0],names[0]

#6)phylogenetic tree
#def differences_matrix(msa):
#create a matrix, then for each pair of aligned sequences in MSA
#use count_diffs[msa[i],msa[j]) to fill mat[i,j] and mat[j,i]
#def newick_step(matrix, newick)
#newick is a list of tuples (name,0)
#i1,i2,new_mat = UPGMA_step(matrix)
#branch = matrix[i1,i2]/2
#n1,b1 = newick[i1]
#n2,b2 = newick[i2]
#new_newick_entry = (f"({n1}:{branch-b1},{n2}:{branch-b2}", branch)
#new_nweick = combine_elements(newick,i1,i2,new_newick_entry)
#return new_mat, new_newick
#def create_newick(msa, names):
#mat=differences_matrix(msa)
#newick=[]
#for n in names:
#  newick.append((n,0))
#while len(newick)>1:
#  mat,newick = newick_step(mat,newick)
#return newick[0][0]+";"

#use tree visualiser to visualise newick data

In [ ]:
#global alignment functions

In [1]:
def global_alignment(s1,s2,gap,match_fn):
    scores,moves=global_alignment_matrix(s1,s2,gap,match_fn)
    a1,a2=traceback(s1,s2,moves)
    return a1,a2,scores[-1,-1]

In [2]:
def traceback(s1,s2,moves):
    align1=""
    align2=""
    r=moves.shape[0]-1
    c=moves.shape[1]-1
    while r>0 or c>0:
        if moves[r,c] ==0:
            align1=s1[r-1]+align1
            align2="-"+align2
            r-=1
        elif moves[r,c] ==1:
            align1="-"+align1
            align2=s2[c-1]+align2
            c-=1
        else:
            align1=s1[r-1]+align1
            align2=s2[c-1]+align2
            r-=1
            c-=1
    return align1, align2

In [3]:
def match_fn(r1,r2):
    if r1==r2:
        return 1
    else:
        return -1

In [4]:
def global_alignment_matrix(s1, s2, gap, match_fn):
#Needleman-Wunsch?
    import numpy as np
    scores = np.zeros((len(s1)+1,len(s2)+1))
    moves = np.zeros((len(s1)+1,len(s2)+1))
    moves[0,:] = 1
    for ix in range(scores.shape[1]):
        scores[0,ix]=gap*ix
    for ix in range(scores.shape[0]):
        scores[ix,0]=gap*ix
    for line in range(1,len(s1)+1):
        for col in range(1,len(s2)+1):
            down=scores[line-1,col]+gap
            right=scores[line,col-1]+gap
            match=scores[line-1,col-1]+match_fn(s1[line-1],s2[col-1])
            scores[line,col] = max((down,right,match))
            moves[line,col] = np.argmax((down,right,match))
    return scores,moves

In [55]:
# def substitution_matrix(seq1, seq2, match=2, mismatch=-1, gapopen=-2, gapextend=-1):
#     s1 = list(max(seq1,seq2))
#     s2 = list(min(seq1,seq2))
#     l1 = len(s1)
#     l2 = len(s2)
    
#     M = [[0]*(l1+1) for i in range(l2+1)]
    
#     for i in range(1,l2+1):
#         for j in range(1,l1+1):
#             if s1[j-1]==s2[i-1]:
#                 M[i][j]=1
#             for row in M:
#                 print(row)
#             print("+++++++++++++++++++++++++++++++++=")
            
#     S = [[0]*(l1+1) for i in range(l2+1)]
    
#     for i in range(1,l2+1):
#         for j in range(1,l1+1):
#             if M[i][j] == 1:
#                 if M[i-1][j-1] == 1:
#                     S[i][j] = S[i-1][j-1]+3
#                 elif M[i-1][j] == 1:
#                     S[i][j] = S[i-1][j]-2
#                 elif M[i][j-1] == 1:
#                     S[i][j] = S[i][j-1]-2
#                 elif S[]:
#                     S[i][j]
#                 elif

#for (i, j) need to check (i-1, j) and (i, j-1) for gap open/extension i.e. take their score and -2 if gap open or -1 if extend or -1 if mismatch
#, and (i-1, j-1) for match+2 or mismatch -1

In [6]:
def count_kmers1(seq, klen, mode="DNA",):
    kmer_dict = {}
    if mode == "DNA":
        DNA = "ACTG"
    elif mode == "protein":
        DNA = "ACDEFGHIKLMNPQRSTVWY*"
    num_kmers = len(seq) - klen +1
    for i in range(num_kmers):
        kmer=seq[i:i+klen]
        if kmer in kmer_dict:
            kmer_dict[kmer] +=1
        else:
            kmer_dict[kmer] =1

    #non_zero = {k:v for k, v in kmer_dict.items() if v != 0}
    return kmer_dict #non_zero

#non_zero = {k:v for k, v in diction.items() if v != 0}
#non_zero = dict(filter(lambda kv: kv[1] != 0 , diction.items()))

In [ ]:
# import time
# s1="CTGTCGTGAATGTACGATAAAAAAAAAAAAAAAACGTAGCTGATGCGGTTTTCTCAGCTACGTTAGTCGTGCATCGAGCATGATCTAATGGGCCCCGGGGCTTCTATTAATCACTATCTTCATTTTGAATTCTTCCTTACACCCCCCTATGATGTATTTCATTCGTAGGAGGGAGAGTCTATAGTATATTTCATTTTTCTAAATCGAAAAAGGTCTCCAACCCAATCGGCCACTACCTAGTCCACCATACTGCACACTCATACTACATCGTACGGGGGGGGTTTTTTTTGCTAAAAAAAAAACCCCCCCCCCCCCCCATCGTACGTACTG"
# s2="TCTTTTCCTTTTTCAGGGCTAGCTAGCATCTTTGAGGTCGATCGTAGCAGCGGGAGAAAAATTCCCCCCCCGAAAAAAAAAAAGCCCCCCCCCCCTAGTCAGCTGACTGGCAGGAGGGTTCTCTCCTGAGGAAATTGAGGGAGAGGAGAGGAGAGAGGGGAGTATACTCTCGTGTGTCCCCCCCCCCTAGCGGGCTAGGCGGTTCTCTTTCTCTGGGGATCGATCGGCATGCTACGTGCATCGATCAGACTTGGGCTCTGCAACATAGATTGACACAAAGTCACGCGCTTTTAGTTACCCCGATCGATCGCCCCCGCCGGTACGTGAAAAAAAAAAAAGC"
# q=list(count_kmers1(s1,5).keys())
# w=list(count_kmers1(s2,5).keys())
# st1 = time.time()
# if set(q).isdisjoint(set(w))==0:
#     print("q")
# st2 = time.time()
# print("set", st2-st1)

# st1 = time.time()
# if any(deca in w for deca in q):
#     print("w")
# st2 = time.time()
# print("comp", st2-st1)


In [ ]:
def decapeptide_filter_query(stats_df, df=False, mode="protein", mer=10):
    if mode=="protein":
        seq_col = "seq"
        kmer_col = "decapeptides"
    else:
        seq_col = "seq"
        kmer_col = "decamers"
        
    stats_df = stats_df.sort_values(by=seq_col, ascending=False).reset_index(drop=True)
    stats_df[kmer_col] = ""
    stats_df_work = stats_df[["id",seq_col,kmer_col,]]
    rep_groups_90 = {}
    rep_groups_95 = {}
    stats_df_work[kmer_col][0] = list(count_kmers1(stats_df_work[seq_col][0], klen=mer, mode=mode).keys())
    rep_set = [pd.DataFrame(stats_df_work.iloc[0]).T]
    
    #while i < stats_df_work.shape[0]:
    for i in range(1,stats_df_work.shape[0]):
        if i%500==0:
            print(i)
        for j in range(len(rep_set)):
            ident_over_90, ident_over_95, rep_kmers, query_kmers = decapeptide_filter(rep_set[j][seq_col][j], rep_set[j]["id"][j], 
                                                                                      stats_df_work[seq_col][i], stats_df_work["id"][i],rep_kmers= rep_set[j][kmer_col][j],mode=mode, w=mer)
            if ident_over_90==True:
                if rep_set[j]["id"][j] not in rep_groups_90:
                    rep_groups_90[rep_set[j]["id"][j]] = [stats_df_work["id"][i]]
                    break
                else:
                    rep_groups_90[rep_set[j]["id"][j]] += [stats_df_work["id"][i]]
                    break
            elif ident_over_95==True:
                if rep_set[j]["id"][j] not in rep_groups_90:
                    rep_groups_95[rep_set[j]["id"][j]] = [stats_df_work["id"][i]]
                    break
                else:
                    rep_groups_95[rep_set[j]["id"][j]] += [stats_df_work["id"][i]]
                    break
        else:
            app = pd.DataFrame(stats_df_work.iloc[i]).T
            app[kmer_col] = [query_kmers]
            app.index = [len(rep_set)]
            rep_set.append(app)

    if df:
        rep_g90 = pd.DataFrame()
        rep_g90["leader"] = rep_groups_90.keys()
        rep_g90["members"] = rep_groups_90.values()
        rep_groups_90 = rep_g90
        #contains group leaders and their corresponding group members

        rep_g95 = pd.DataFrame()
        rep_g95["leader"] = rep_groups_95.keys()
        rep_g95["members"] = rep_groups_95.values()
        rep_groups_95 = rep_g95
        #contains group leaders and their corresponding group members
        
        rep_df = pd.concat(rep_set)
        rep_set=rep_df
        #contains group leaders and singletons, and their decamer breakdown
    
    return rep_groups_90, rep_groups_95, rep_set

In [ ]:
def decapeptide_filter(rep_seq, rid, query, qid, w=10, rep_kmers=False, mode="protein",):
    #90% threshold
    seq_len = min(len(query),len(rep_seq))
    
    ident_over_90=False
    ident_over_95=False
    
    query_kmers = list(count_kmers1(query, klen=w, mode=mode).keys())
    if rep_kmers==False:
        rep_kmers = list(count_kmers1(rep_seq, klen=w, mode=mode).keys())
    if any(deca in rep_kmers for deca in query_kmers):
        if seq_len<20:
            ident_over_95=True
        else:
            ident_over_90=True
    
    return ident_over_90, ident_over_95, rep_kmers, query_kmers

In [84]:
def ktuple_filter(seq1, seq2, w=10, k=10, thresh_pident=90):
    thresh_match = int(k/100*thresh_pident)+1
    seq_len = min(len(seq1),len(seq2))
    mismatches_allowed = 100-(int(seq_len/100*thresh_pident)+1)
    a = int(seq_len/w)
    b = seq_len%w
    num_ktuples = a-mismatches_allowed
    min_ktuples = w*num_ktuples-(w-1)
    thresh_match = seq_len/100*thresh_pident
    return mismatches_allowed, a, b, num_ktuples, min_ktuples

# import scipy.special
# b=0
# L=100
# w=5
# k=5
# p=k/w-1/w+b/w*(w-k+1)/L
# wk_binom = scipy.special.binom(10,10)
# T=(L-w+1)*wk_binom+w-k # number of ktuples covering the query
# m=n*k*wk_binom # number of mismatching ktuples
# M = w-k+(L-w+1-(1-p)*L*k)*wk_binom # number of matching ktuples at given k,w,p,and L

In [15]:
def import_df_by_part2(df_part_dir):
    import os
    import glob
    import pandas as pd
    import numpy as np
    i=0
    pathlist = glob.glob(os.path.join(df_part_dir,"*"))
    for infile in sorted(pathlist):
        if i==0:
            fin=pd.read_csv(infile, sep="\t")
            icols = fin.select_dtypes("integer").columns
            fin[icols] = fin[icols].apply(lambda x: pd.to_numeric(x, downcast="integer"))
            i+=1
        else:
            p=pd.read_csv(infile, sep="\t")
            icols = p.select_dtypes("integer").columns
            p[icols] = p[icols].apply(lambda x: pd.to_numeric(x, downcast="integer"))
            fin=pd.concat([fin,p],axis=0,ignore_index=True)
            fin = fin.fillna(0)
    return fin

In [31]:
def import_df_by_part1(df_part_dir):
    from pathlib import Path
    pathlist = Path(df_part_dir).glob("**/*")
    for path in pathlist:
        path_in_str=str(path)
        print(path_in_str)

In [25]:
def import_df_by_part(df_part_dir):
    import os
    dir = os.fsencode(df_part_dir)
    for file in os.listdir(dir):
        fname=os.fsdecode(file)
        print(fname)

In [12]:
def substring_coords(seq, subseq):
    # "TATAGCATATC" returns [1] [4]
    count = start = 0
    starts = []
    while True:
        start = seq.find(subseq, start) + 1
        if start > 0:
            starts.append(start)
        else:
            ends = [x+len(subseq)-1 for x in starts]
            return starts, ends

In [185]:
def substring_occurrences(seq, subseq):
    count = start = 0
    while True:
        start = seq.find(subseq, start) + 1
        if start > 0:
            count += 1
        else:
            return count

In [188]:
def search_polyA(seq, min_polyA = 15, rev_comp=False, get_seq=False, contains=True, polyA_list=[], polyA_pattern=[]):
    #rev_comp - whether to search for polyA via reverse complementing the sequence - useless with kmer approach
    #get_seq - return sequence
    #contains - performs any() before doing regex search. more speed benefit when there are many negatives
    #polyA_list - list of generated polyAs, directly used for contains segment, can be filled with anything random if contains is False, but cant be left empty if kmers used
    #polyA_pattern - pattern containing all kmers generated by "generate_impure_pA" function
    false_positive=True
    seq_len = len(seq)
    import re
    polyA_re = r"\1"*(min_polyA-2)
    pattern = fr"([A]+?){polyA_re}\1+"
    pA = re.finditer(pattern,seq)
    pattern2 = fr"([T]+?){polyA_re}\1+"
    pA2 = re.finditer(pattern2,seq)
    starts = []
    ends = []
    pA_lens = []
    rev_compd=False
    non_consecutive=False
    if pA:
        for i in pA:
            starts.append(i.start())
            ends.append(i.end())
            pA_lens.append(len(i[0]))
    if pA2:
        for i in pA2:
            starts.append(i.start())
            ends.append(i.end())
            pA_lens.append(len(i[0]))
    if len(starts) == 0:
        if rev_comp == True:
            polyA_re = r"\1"*(min_polyA-2)
            pattern = fr"([A]+?){polyA_re}\1+"
            seq = rev_comp(seq,mode="rev_comp")[0]
            pA = re.finditer(pattern,seq)
            starts = []
            ends
            pA_lens = []
            rev_compd = True
            for i in pA:
                starts.append(i.start())
                ends.append(i.end())
                pA_lens.append(len(i[0]))
        if len(starts) == 0:
            if len(polyA_list)>0:
                if rev_comp == True:
                    polyA_re = r"\1"*(min_polyA-2)
                    pattern = fr"([A|T]+?){polyA_re}\1+"
                    seq = rev_comp(seq,mode="rev_comp")[0]
                    pA = re.finditer(pattern,seq)
                elif contains == True:
                    if any(pAgen in seq for pAgen in polyA_list):
                        pA = re.finditer(polyA_pattern, seq)
                else:
                    pattern = fr"({polyA_pattern})"
                    pA = re.finditer(pattern,seq)
            starts = []
            ends = []
            pA_lens = []
            for i in pA:
                starts.append(i.start())
                ends.append(i.end())
                pA_lens.append(len(i[0]))
            non_consecutive = True
            if len(starts)>0:
                false_positive=True
                if min(starts) <150:
                    false_positive = False
                    rev_compd=True
                if max(ends)>seq_len-150:
                    false_positive = False

        else:
            false_positive=True
            if min(starts) <150:
                false_positive = False
                rev_compd=True
            if max(ends)>seq_len-150:
                false_positive = False

    else:
        false_positive=True
        if min(starts) <150:
            false_positive = False
            rev_compd=True
        if max(ends)>seq_len-150:
            false_positive = False

    if get_seq == False:
        seq=""
        
    return starts, ends, pA_lens, rev_compd, seq, non_consecutive, seq_len, false_positive
    #starts - start coords
    #ends - end coords
    #pA_lens - pA lengths
    #rev_compd - whether it was required to rev_comp the sequence to find the pA
    #seq - sequence, will be rev_comped if it was to find pA
    #non_consecutive - whether it was necessary to use pre-generated impure kmers to find the pA
    #false_positive - whether the pA was found at one of the ends of the sequence or in the middle (false positie)
#example:
#pA_starts, pA_ends, pA_lens, pA_ev_compd, pA_seq, pA_non_consecutive, pA_seq_len, pA_false_positive = search_polyA(seq, contains=True, polyA_list=pA_gen, polyA_pattern=pA_pattern)

In [187]:
def generate_impure_pA(chain=15, impure=1, pattern=True):
    alphabet = ["A","C","T","G"]
    pure_a = ["A", "T"]
    seq_list = []

    if impure == 1:
        for letter in pure_a:
            seq_list.append(letter*chain)
            alph = [x for x in alphabet if x not in [letter]]
            for impurity in alph:
                for i in range(chain):
                    next_seq = list(letter*chain)
                    next_seq[i] = impurity
                    next_seq = "".join(next_seq)
                    seq_list.append(next_seq)

    if impure == 2:
        for letter in pure_a:
            seq_list.append(letter*chain)
            alph = [x for x in alphabet if x not in [letter]]
            for impurity in alph:
                for i in range(chain):
                    for j in range(i,chain):
                        matrix = [0]*chain
                        if i==j:
                            z = i
                        matrix[j]=1
                        matrix[z]=1
                        next_seq = list(map(lambda x: impurity if x != 0 else letter, matrix))
                        next_seq = "".join(next_seq)
                        seq_list.append(next_seq)

    if pattern == True:
        pattern = ""
        for i in seq_list:
            pattern += i
            pattern += "|"
        pattern = pattern[:-1]

    return seq_list, pattern

In [2]:
def find_tr_ORF_stats(seq_list, seq_ids, save_path=False, batch_size=10000,pA_chain_gen=[20,2], check_MREs=True, include_seq=False,tissue=False):
    import time
    start = time.time()
    tr_stats = []
    i=0
    j=0
    z=0
    pA_gen, pA_pattern = generate_impure_pA(chain=pA_chain_gen[0], impure=pA_chain_gen[1])
    
    for seq_idx in range(len(seq_list)):
        j+=1
        if j%batch_size == 0:
            print(j)
        if save_path!=False:
            if j%batch_size == 0:
                tr_stats = pd.concat(tr_stats, axis=0)
                tr_stats = tr_stats.reset_index(drop=True)
                tr_stats.to_csv(save_path+f"_{j}", index=False, sep="\t")
                i=0
        out = find_ORFs(seq_list[seq_idx], len_filter=26, stops=["TAA","TGA","TAG"], to_translate=True)
        if len(out[3])>0:
            stats_df, cols = flnc_tr_seq_stats_no_pd(seq_list[seq_idx], seq_ids[seq_idx], ORF_coords=out[5],polyA_pattern=pA_pattern, polyA_list=pA_gen,check_MREs=check_MREs, all_ORFs=out[0], ORFs_out=out, include_seq=include_seq,tissue=tissue)
            # stats_df = pd.DataFrame([[x for x in stats_df]])
            # stats_df = pd.DataFrame([stats_df])
            if i == 0:
                tr_stats = []
                tr_stats.append(stats_df)
                # tr_stats = stats_df.copy()
                i+=1
            else:
                tr_stats.append(stats_df)
        else:
            if z==0:
                print("No ORFs found#")
                #stats_df = pd.DataFrame()
                z+=1
            else:
                print("No ORFs found")
                #stats_df["num_ORFs_found"] = 0
                
    if save_path!=False:
        if j%batch_size == 0:
            tr_stats = pd.concat(tr_stats, axis=0)
            tr_stats = tr_stats.reset_index(drop=True)
            tr_stats.to_csv(save_path+f"_{j}", index=False, sep="\t")
            i=0
    else:
        tr_stats = pd.DataFrame(tr_stats)
        tr_stats.columns = cols
        # tr_stats = pd.concat(tr_stats,axis=0)
        
    end = time.time()
    print(end-start)
    return tr_stats

In [107]:
def which_frame(seq, subseq):
    substring_starts = []
    i=0
    while i != -1:
        pi = i
        i = seq[i:].find(subseq)
        if i != -1: 
            i = pi + i + 1
            substring_starts.append(i)
    frames=[]
    if len(substring_starts) == 0:
        print("No subseq match found")
    else:
        for start in substring_starts:
            f = start%3
            if f == 0:
                frame = 3
            elif f == 1:
                frame = 1
            elif f == 2:
                frame = 2
            frames.append(frame)
    return frames, substring_starts

In [186]:
def which_frame_by_coord(start_coords):
    #zero indexed coords from re.finditer
    frames=[]
    if len(start_coords) == 0:
        print("No coords present")
    else:
        for start in start_coords:
            f = start%3
            if f == 0:
                frame = 1
            elif f == 1:
                frame = 2
            elif f == 2:
                frame = 3
            frames.append(frame)
    return frames
#can implement an option to extract subseqs if end or len is supplied

In [1705]:
def flnc_tr_seq_stats_no_pd(seq, seq_id="placeholder", ORF_coords=[], stops=["TAA","TAG","TGA"], min_polyA=15, seq_store=False, polyA_list=[],polyA_pattern=[], ORFs_out=[], check_MREs=True, all_ORFs=[], include_seq=False, tissue=False):
    seq_len = len(seq)
    stop_num = 0
    stop_count = {}
    for codon in stops:
        i = substring_occurrences(seq,codon)
        stop_count[codon] = i #stop counts dict
        stop_num += i #total stops
 
    G = seq.count("G")
    C = seq.count("C")
    GC = 100/seq_len*(G+C) #GC

    ATG_num = substring_occurrences(seq,"ATG") #starts
    seq_start_ratio = ATG_num/seq_len #start ratio
    seq_stop_ratio = stop_num/seq_len #stop ratio
    
    TGT_num = substring_occurrences(seq, "TGT") #cys1
    TGC_num = substring_occurrences(seq, "TGC") #cys2
    seq_TGT_ratio = TGT_num/seq_len #
    seq_TGC_ratio = TGC_num/seq_len #
    Cys_codon_ratio = seq_TGT_ratio+seq_TGC_ratio #

    tissue_id=None
#    if include_seq:
#        stats_df["seq"] = seq #
    if tissue:
        tissues = ["crop", "cal_g", "neph", "nerve_c", "clitel", "bod_w", "phar", "gut", "gizz", "sem_v"] 
        for t in tissues:
            if t in seq_id:
                tissue_id = t #tissue id
                break
            
    #can be optimised by merging with ORF code
    #can also be expanded by splitting by stop types 
    frames, locs = which_frame(seq, "ATG")
    frame_starts = [0]*3
    for frame in frames:
        frame_starts[frame-1] += 1 # frame start counts by frame

    frame_stops = [0]*3
    for stop in stops:
        frames, locs = which_frame(seq, stop)
        for frame in frames:
            frame_stops[frame-1] += 1 # frame stop counts by frame
            
    if len(ORF_coords)>0: # Largest_ORF_coords all_ORFs # All ORF coords
        ORF_stop = seq[ORF_coords[1]-3:ORF_coords[1]] # ORF_stop_seq
        ORF_frame = ORF_coords[0]%3
        if ORF_frame == 0:
            ORF_frame = 3
        elif ORF_frame == 1:
            ORF_frame = 1
        elif ORF_frame == 2:
            ORF_frame = 2 #largest ORF frame
        
        ORF_seq = seq[ORF_coords[0]-1:ORF_coords[1]]
        all_ORF_coords = all_ORFs
        p5_seq = seq[0:ORF_coords[0]]
        p3_seq = seq[ORF_coords[1]-1:seq_len]
        seq_list = [p5_seq, ORF_seq, p3_seq]
        prot_seq = translate(ORF_seq) # ORF prot
        all_prot_seq=[]
        all_prot_cys = 0
        for orf in all_ORF_coords:
            p = translate(seq[orf[0]-1:orf[1]])
            all_prot_seq.append(p)
            all_prot_cys += p.count("C")
        prot_len = len(prot_seq) # ORF prot leb
        Cys_num = prot_seq.count("C") # cys num
        if Cys_num>0:
            cys_ratio = Cys_num/prot_len # prot cys 
        else:
            cys_ratio = 0
        
        ORF_seq_len = len(ORF_seq)
        p5_seq_len = len(p5_seq)
        p3_seq_len = len(p3_seq)
        region_lens = [p5_seq_len, ORF_seq_len, p3_seq_len] # region lens

        ORF_ATG_num = substring_occurrences(ORF_seq,"ATG") # ORF_ATG_total
        ORF_TGT_num = substring_occurrences(ORF_seq,"TGT") # ORF_TGT_total
        ORF_TGC_num = substring_occurrences(ORF_seq,"TGC") # ORF_TGC_total
        ORF_TGT_ratio = ORF_TGT_num/ORF_seq_len # ORF_TGT_ratio
        ORF_TGC_ratio = ORF_TGC_num/ORF_seq_len # ORF_TGC_ratio
        ORF_Cys_ratio = ORF_TGT_ratio+ORF_TGC_ratio # ORF_Cys_ratio
        
        p5_ATG_num = substring_occurrences(p5_seq,"ATG") 
        p3_ATG_num = substring_occurrences(p3_seq,"ATG") 
        ATG_list = [p5_ATG_num, ORF_ATG_num, p3_ATG_num] # region_start_nums
        
        regions = ["p5", "ORF", "p3"]
        region_stops = {}
        region_stop_ratios = []
        region_start_ratios = []
        for region in range(len(seq_list)):
            r_stop_counts = {}
            r_stop_num = 0
            for codon in stops:
                i = substring_occurrences(seq_list[region],codon)
                r_stop_counts[codon] = i
                r_stop_num += i
            r_stop_counts["total"] = r_stop_num
            region_stops[regions[region]] = r_stop_counts #region_stop_stats

            region_stop_ratios.append(region_stops[regions[region]]["total"]/region_lens[region]) #region_stop_ratios
            region_start_ratios.append(ATG_list[region]/region_lens[region]) #region_start_ratios
    
    pA_Starts, pA_Ends, pA_Lengths, Rev_Compd, Seq, Non_Consecutive, Seq_Len, False_Positive = search_polyA(seq, polyA_list=polyA_list, polyA_pattern=polyA_pattern)
    
    spans = []
    pA_lens = []
    
    for i in range(len(pA_Starts)):
        spans.append([pA_Starts[i],pA_Ends[i]])
        pA_lens.append(pA_Lengths[i])

    if len(spans) == 0:
        polyA_span=None # polyA_span
        polyA_len=None # polyA_len
        pA_rev_compd=Rev_Compd # pA_rev_compd
        pA_non_consecutive=Non_Consecutive # pA_non_consecutive
        pA_false_positive = False_Positive # pA_false_positive
    else:
        polyA_span=spans
        polyA_len=pA_lens
        pA_rev_compd=Rev_Compd
        pA_non_consecutive=Non_Consecutive
        pA_false_positive = False_Positive
    pA_sig_span = []
    
    if len(pA_lens)>0:
        pA_start = sorted([x for i in spans for x in i])[-2]
        pA_sig_region = seq[pA_start-100:pA_start]
        pA_sig_pattern = "AATAAA"

        pA_sig_span = []
        q=0
        while q != -1:
            pq = q
            q = pA_sig_region[q:].find(pA_sig_pattern)
            if q != -1: 
                q = pq + q + 1
                g = q+pA_start-100
                b = g+6
                pA_sig_span.append([g,b]) #polyA_signal_span
        
    if check_MREs==True:
        MREs_list = ["TGCGCAC","TGCGCGC","TGCGCCC","TGCGCTC","TGCACAC","TGCACGC","TGCACCC","TGCACTC","GAGTGCA","GGGTGCA","GCGTGCA","GTGTGCA","GAGCGCA","GGGCGCA","GCGCGCA","GTGCGCA"]
        MRE_starts = []
        MRE_types = []
        MRE_frames = []
        for MRE in MREs_list:
            q=0
            while q != -1:
                pq = q
                q = seq[q:].find(MRE)
                if q != -1: 
                    q = pq + q + 1
                    MRE_starts.append(q) #MRE_starts
                    MRE_types.append(MRE) #MRE_types
                    MRE_frames.append(which_frame_by_coord([q])) #MRE_frames

        MRE_number = len(MRE_starts) #MRE_number
        MREs = list(zip(MRE_starts,MRE_types)) #MREs
    
    num_ORFs_found = ORFs_out[6]
    lens_above_filter = ORFs_out[3]
    prots =ORFs_out[4]
    stops_above_filter = ORFs_out[2]
    
    cols = ["id", "tissue", "seq", "seq_len", "GC", "stop_counts", "total_stops", "ATG_num", "seq_start_ratio", "seq_stop_ratio", "TGT_num", "TGC_num", "seq_TGT_ratio","seq_TCT_ratio", "cys_codon_ratio", "starts_per_frame", "stops_per_frame", "ORF_coords", "all_ORF_coords", "num_ORFs_found", "ORF_lengths", "all_prots", "ORF_stop", "all_ORF_stops", "ORF_frame", "prot_seq", "prot_len", "cys_num", "cys_ratio", "region_lengths", "ORF_ATG_num", "ORF_TGT_num", "ORF_TGC_num", "ORF_TGT_ratio", "ORF_TGC_ratio", "ORF_cys_ratio", "region_ATGs", "region_stops", "region_stop_ratios", "region_start_ratios", "polyA_span", "polyA_len", "pA_rev_compd", "pA_non_consecutive", "pA_false_positive", "pA_sig_span", "MRE_starts", "MRE_types", "MRE_frames", "MRE_number", "MREs"]
    out = [ seq_id, tissue_id, seq, seq_len, GC, stop_count, stop_num, ATG_num, seq_start_ratio, seq_stop_ratio, TGT_num, TGC_num, seq_TGT_ratio, seq_TGC_ratio, Cys_codon_ratio, frame_starts, frame_stops, ORF_coords, all_ORF_coords, num_ORFs_found, lens_above_filter, prots, ORF_stop, stops_above_filter, ORF_frame, prot_seq, prot_len, Cys_num, cys_ratio, region_lens, ORF_ATG_num, ORF_TGT_num, ORF_TGC_num, ORF_TGT_ratio, ORF_TGC_ratio, ORF_Cys_ratio, ATG_list, region_stops, region_stop_ratios, region_start_ratios, polyA_span, polyA_len, pA_rev_compd, pA_non_consecutive, pA_false_positive, pA_sig_span, MRE_starts, MRE_types, MRE_frames, MRE_number, MREs]
    return out, cols

49

In [193]:
def flnc_tr_seq_stats(seq, seq_id="placeholder", ORF_coords=[], stops=["TAA","TAG","TGA"], min_polyA=15, seq_store=False, polyA_list=[],polyA_pattern=[], check_MREs=True, all_ORFs=[], include_seq=False, tissue=False):
    import pandas as pd
    import re
    pd.set_option("display.max_columns", None)
    stats_df = pd.DataFrame([1])
    seq_len = len(seq)
    stop_num = 0
    stop_count = {}
    for codon in stops:
        i = substring_occurrences(seq,codon)
        stop_count[codon] = i
        stop_num += i
        stats_df[f"{codon}_total"] = i

    G = seq.count("G")
    C = seq.count("C")
    GC = 100/seq_len*(G+C)
    stats_df["GC"] = GC

    ATG_num = substring_occurrences(seq,"ATG")
    seq_start_ratio = ATG_num/seq_len
    seq_stop_ratio = stop_num/seq_len
    
    TGT_num = substring_occurrences(seq, "TGT")
    TGC_num = substring_occurrences(seq, "TGC")
    seq_TGT_ratio = TGT_num/seq_len
    seq_TGC_ratio = TGC_num/seq_len
    Cys_codon_ratio = seq_TGT_ratio+seq_TGC_ratio

    if seq_store == "all":
        stats_df["seq"] = seq

    stats_df["id"] = seq_id
    stats_df["seq_len"] = seq_len
    stats_df["ATG_total"] = ATG_num
    stats_df["seq_start_ratio"] = seq_start_ratio
    stats_df["seq_stop_ratio"] = seq_stop_ratio   
    stats_df["TGT_total"] = TGT_num
    stats_df["TGC_total"] = TGC_num
    stats_df["seq_TGT_ratio"] = seq_TGT_ratio
    stats_df["seq_TGC_ratio"] = seq_TGC_ratio  
    stats_df["Cys_codon_ratio"] = Cys_codon_ratio
    
    if include_seq:
        stats_df["seq"] = seq
        if tissue:
            stats_df["tissue"] = ""
            tissue_regex = r'(crop|cal_g|neph|nerve_c|clitel|bod_w|phar|gut|gizz|sem_v)'
            stats_df["tissue"] = stats_df["id"].str.extract(tissue_regex, expand=False)

    #can be optimised by merging with ORF code
    #can also be expanded by splitting by stop types 
    frames, locs = which_frame(seq, "ATG")
    frame_starts = [0]*3
    for frame in frames:
        frame_starts[frame-1] += 1
        
    frame_stops = [0]*3
    for stop in stops:
        frames, locs = which_frame(seq, stop)
        for frame in frames:
            frame_stops[frame-1] += 1
            
    stats_df["start_counts_by_frame"] = [frame_starts]
    stats_df["stop_counts_by_frame"] = [frame_stops]
    
    if len(ORF_coords)>0:
        stats_df["ORF_coords"] = [ORF_coords]
        stats_df["ORF_stop"] = seq[ORF_coords[1]-3:ORF_coords[1]]
        stats_df["all_ORF_coords"] = [all_ORFs]
        ORF_frame = ORF_coords[0]%3
        if ORF_frame == 0:
            ORF_frame = 3
        elif ORF_frame == 1:
            ORF_frame = 1
        elif ORF_frame == 2:
            ORF_frame = 2
        stats_df["ORF_frame"] = ORF_frame
        
        ORF_seq = seq[ORF_coords[0]-1:ORF_coords[1]]
        p5_seq = seq[0:ORF_coords[0]]
        p3_seq = seq[ORF_coords[1]-1:seq_len]
        seq_list = [p5_seq, ORF_seq, p3_seq]
        if seq_store == "all" or seq_store=="regions":
            stats_df["region_seqs"] = [seq_list]
        prot_seq = translate(ORF_seq)
        stats_df["ORF_prot"] = prot_seq
        prot_len = len(prot_seq)
        stats_df["ORF_prot_len"] = prot_len
        Cys_num = prot_seq.count("C")
        stats_df["cys_num"] = Cys_num
        if Cys_num>0:
            cys_ratio = Cys_num/prot_len
            stats_df["cys_ratio"] = cys_ratio
        else:
            stats_df["cys_ratio"] = 0

        ORF_seq_len = len(ORF_seq)
        p5_seq_len = len(p5_seq)
        p3_seq_len = len(p3_seq)
        len_list = [p5_seq_len, ORF_seq_len, p3_seq_len]
        stats_df["region_lens"] = [len_list]

        ORF_ATG_num = substring_occurrences(ORF_seq,"ATG")
        ORF_TGT_num = substring_occurrences(ORF_seq,"TGT")
        ORF_TGC_num = substring_occurrences(ORF_seq,"TGC")
        ORF_TGT_ratio = ORF_TGT_num/ORF_seq_len
        ORF_TGC_ratio = ORF_TGC_num/ORF_seq_len
        ORF_Cys_ratio = ORF_TGT_ratio+ORF_TGC_ratio
        
        p5_ATG_num = substring_occurrences(p5_seq,"ATG")
        p3_ATG_num = substring_occurrences(p3_seq,"ATG")
        ATG_list = [p5_ATG_num, ORF_ATG_num, p3_ATG_num]
        stats_df["region_start_nums"] = [ATG_list]
        stats_df["ORF_TGT_total"] = ORF_TGT_num
        stats_df["ORF_TGC_total"] = ORF_TGC_num
        stats_df["ORF_TGT_ratio"] = ORF_TGT_ratio
        stats_df["ORF_TGC_ratio"] = ORF_TGC_ratio
        stats_df["ORF_Cys_ratio"] = ORF_Cys_ratio
        
        regions = ["p5", "ORF", "p3"]
        region_stops = {}
        
        for region in range(len(seq_list)):
            r_stop_counts = {}
            r_stop_num = 0
            for codon in stops:
                i = substring_occurrences(seq_list[region],codon)
                r_stop_counts[codon] = i
                r_stop_num += i
            r_stop_counts["total"] = r_stop_num
            region_stops[regions[region]] = r_stop_counts

            stats_df[f"{regions[region]}_stop_ratio"] = region_stops[regions[region]]["total"]/len_list[region]
            stats_df[f"{regions[region]}_start_ratio"] = ATG_list[region]/len_list[region]
        stats_df["region_stop_stats"] = [region_stops]

    pA_Starts, pA_Ends, pA_Lengths, Rev_Compd, Seq, Non_Consecutive, Seq_Len, False_Positive = search_polyA(seq, polyA_list=polyA_list, polyA_pattern=polyA_pattern)
    
    spans = []
    pA_lens = []
    
    for i in range(len(pA_Starts)):
        spans.append([pA_Starts[i],pA_Ends[i]])
        pA_lens.append(pA_Lengths[i])

    if len(spans) == 0:
        stats_df["polyA_span"] = None
        stats_df["polyA_len"] = None
        stats_df["pA_rev_compd"] = Rev_Compd
        stats_df["pA_non_consecutive"] = Non_Consecutive
        stats_df["pA_false_positive"] = False_Positive
    else:
        stats_df["polyA_span"] = [spans]
        stats_df["polyA_len"] = [pA_lens]
        stats_df["pA_rev_compd"] = Rev_Compd
        stats_df["pA_non_consecutive"] = Non_Consecutive
        stats_df["pA_false_positive"] = False_Positive

    if len(pA_lens)>0:
        pA_start = sorted([x for i in spans for x in i])[-2]
        pA_sig_region = seq[pA_start-100:pA_start]
        pA_sig_pattern = r"((AATAAA)+?)"
        pA_sig = re.finditer(pA_sig_pattern, pA_sig_region)
        pA_sig_span = []
        for i in pA_sig:
            pA_sig_coords = list(i.span())
            pA_sig_left = pA_sig_coords[0]+pA_start-100
            pA_sig_right = pA_sig_coords[1]+pA_start-100
            pA_sig_span.append([pA_sig_left,pA_sig_right])
        stats_df["polyA_signal_span"] = [pA_sig_span]
        
    if check_MREs==True:
        stats_df["MREs"] = [[[i.start(),i.group(0)] for i in re.finditer("TGCGCAC|TGCGCGC|TGCGCCC|TGCGCTC|TGCACAC|TGCACGC|TGCACCC|TGCACTC|GAGTGCA|GGGTGCA|GCGTGCA|GTGTGCA|GAGCGCA|GGGCGCA|GCGCGCA|GTGCGCA", seq)]]
        stats_df["MRE_starts"]=stats_df["MREs"].apply(lambda x: [i[0] for i in x])
        stats_df["MRE_types"]=stats_df["MREs"].apply(lambda x: [i[1] for i in x])
        stats_df["MRE_frames"]=stats_df["MRE_starts"].apply(lambda x:which_frame_by_coord(x))
        stats_df["MRE_starts_sorted"]=stats_df["MRE_starts"].apply(lambda x: sorted(x))
        stats_df["MRE_number"]=stats_df["MRE_starts"].apply(lambda x: len(x))
    
    return stats_df

In [ ]:
def prot_properties(seq, table="simple"):
    if table == "simple":
        table = {"R":"P", "H":"P", "K":"P", 
            "D":"P", "E":"P", 
            "S":"P", "T":"P", "N":"P", "Q":"P", 
            "A":"H", "V":"H", "I":"H", "L":"H", "M":"H", "F":"H", "Y":"H", "W":"H",
            "C": "*", "G":"H", "P":"H"}
        
    elif table =="charged":
        table = {"R":"+", "H":"+", "K":"+", 
         "D":"-", "E":"-", 
         "S":"P", "T":"P", "N":"P", "Q":"P", 
         "A":"H", "V":"H", "I":"H", "L":"H", "M":"H", "F":"H", "Y":"H", "W":"H",
         "C": "*", "G":"H", "P":"H"}
    
    elif table =="MirnyShakhnovich": #aliphatic A #aromatic R #polar P #positive + #negative - #hydrophobic H
        table = {"R":"+", "K":"+", 
         "D":"-", "E":"-", 
         "S":"P", "T":"P", "N":"P", "Q":"P",         
         "A":"A", "V":"A", "L":"A", "I":"A", "M":"H", "C":"A",
         "F":"R", "Y":"R", "W":"R", "H":"R",
         "G":"H", "P":"H"}

    prop_seq = ""
    for i in range(0, len(seq)):
        aa = seq[i]
        prop_seq += table[aa]
    return prop_seq

In [15]:
def rev_comp(seq, mode):
    table = {"A":"T", "T":"A", "C":"G", "G":"C"}
    out = []
    if mode == "rev_comp" or mode == "all":
        seq_rev = seq[::-1]
        seq_rev_comp = ""
        for i in range(0,len(seq)):
            nt = seq_rev[i]
            seq_rev_comp += table[nt]
        out.append(seq_rev_comp)

    if mode == "comp" or mode == "all":
        seq_rev_comp = ""
        for i in range(0,len(seq)):
            nt = seq[i]
            seq_rev_comp += table[nt]
        out.append(seq_rev_comp)
        
    if mode == "rev" or mode == "all":
        seq_rev = seq[::-1]
        out.append(seq_rev)
        
    return out

In [210]:
def translate(seq, frame=0, table="standard", stops=[]):
    '''
    frame 0-2 = frames 1-3 fwd
    frame 3-5 = frames 4-6 rev
    frame 6-8 = frames 7-9 comp
    frame 9-11 = frames 10-12 rev_comp
    frame 12 = all 1-3
    frame 13 = all 4-6
    frame 14 = all 7-9
    frame 15 = all 10-12
    frame 16 = all 1-12
    '''
    
    if len(stops)>0:
        table_dict = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L", "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "TAT":"Y", "TAC":"Y",
                          "TAA":"Y", "TAG":"G", "TGT":"C", "TGC":"C", "TGA":"W", "TGG":"W", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
                          "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P", "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", "CGT":"R", "CGC":"R",
                          "CGA":"R", "CGG":"R", "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
                          "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K", "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R", "GTT":"V", "GTC":"V",
                          "GTA":"V", "GTG":"V", "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A", "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
                          "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G"}
        for stop in stops:
            table_dict[stop] = "*"
    elif table == "standard":
        table_dict = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L", "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "TAT":"Y", "TAC":"Y",
                      "TAA":"*", "TAG":"*", "TGT":"C", "TGC":"C", "TGA":"*", "TGG":"W", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
                      "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P", "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", "CGT":"R", "CGC":"R",
                      "CGA":"R", "CGG":"R", "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
                      "AAT":"N", "AAC":"N", "AAA":"K", "AAG":"K", "AGT":"S", "AGC":"S", "AGA":"R", "AGG":"R", "GTT":"V", "GTC":"V",
                      "GTA":"V", "GTG":"V", "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A", "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
                      "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G"}
    elif table == "flatworm":
        table_dict = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L", "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "TAT":"Y", "TAC":"Y",
                      "TAA":"Y", "TAG":"*", "TGT":"C", "TGC":"C", "TGA":"W", "TGG":"W", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
                      "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P", "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", "CGT":"R", "CGC":"R",
                      "CGA":"R", "CGG":"R", "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
                      "AAT":"N", "AAC":"N", "AAA":"N", "AAG":"K", "AGT":"S", "AGC":"S", "AGA":"S", "AGG":"S", "GTT":"V", "GTC":"V",
                      "GTA":"V", "GTG":"V", "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A", "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
                      "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G"}
    elif table == "TAA":
        table_dict = {"TTT":"F", "TTC":"F", "TTA":"L", "TTG":"L", "TCT":"S", "TCC":"S", "TCA":"S", "TCG":"S", "TAT":"Y", "TAC":"Y",
                      "TAA":"*", "TAG":"G", "TGT":"C", "TGC":"C", "TGA":"W", "TGG":"W", "CTT":"L", "CTC":"L", "CTA":"L", "CTG":"L",
                      "CCT":"P", "CCC":"P", "CCA":"P", "CCG":"P", "CAT":"H", "CAC":"H", "CAA":"Q", "CAG":"Q", "CGT":"R", "CGC":"R",
                      "CGA":"R", "CGG":"R", "ATT":"I", "ATC":"I", "ATA":"I", "ATG":"M", "ACT":"T", "ACC":"T", "ACA":"T", "ACG":"T",
                      "AAT":"N", "AAC":"N", "AAA":"N", "AAG":"K", "AGT":"S", "AGC":"S", "AGA":"S", "AGG":"S", "GTT":"V", "GTC":"V",
                      "GTA":"V", "GTG":"V", "GCT":"A", "GCC":"A", "GCA":"A", "GCG":"A", "GAT":"D", "GAC":"D", "GAA":"E", "GAG":"E",
                      "GGT":"G", "GGC":"G", "GGA":"G", "GGG":"G"}

    prot = ""
    prot_seq = ""
    done = ""
    if frame < 3:
        done = "true"
    elif frame <6:
        seq = seq[::-1]
        done = "true"
    elif frame < 9:
        seq = rev_comp(seq, "comp")[0]
        done = "true"
    elif frame < 12:
        seq = rev_comp(seq, "rev_comp")[0]
        done = "true"
        
    if done == "true":
        leftover = (len(seq)-frame)%3
        for i in range(0+frame, len(seq)-leftover, 3):
            codon = seq[i:i+3]
            prot += table_dict[codon]
    else:
        prot = []
        if frame < 13:
            done = "true"
        elif frame < 14:
            seq = seq[::-1]
            done = "true"
        elif frame < 15:
            seq = rev_comp(seq, "comp")[0]
            done = "true"
        elif frame < 16:
            seq = rev_comp(seq, "rev_comp")[0]
            done = "true"

        if done == "true":
            for j in range(0,3):
                prot_seq=""
                leftover = (len(seq)-j)%3
                for i in range(0+j, len(seq)-leftover, 3):
                    codon = seq[i:i+3]
                    prot_seq += table_dict[codon]
                prot+=[prot_seq]
            
        elif frame < 17:
            for s in [seq, seq[::-1], rev_comp(seq, "comp")[0], rev_comp(seq, "rev_comp")[0]]:
                for j in range(0,3):
                    prot_seq=""
                    leftover = (len(s)-j)%3
                    for i in range(0+j, len(s)-leftover, 3):
                        codon = s[i:i+3]
                        prot_seq += table_dict[codon]
                    prot+=[prot_seq]
        else:
            print("provide a valid frame option")

    return(prot)

In [ ]:
def read_blast(path, self=False, header=0, sep="\t"):
    if header != 0:
        blast_df = pd.read_csv(path, sep=sep, header=header)
    else:
        blast_df = pd.read_csv(path, sep=sep)
    if blast_df.shape[0] != 0:        
        blast_df.columns = ["qseqid", "sseqid", "slen", "qlen", "qstart", "qend", "sstart", "send", "length", "mismatch", "gapopen", "pident", "evalue", "bitscore"]
        blast_df["sorientation"] = "+"
        blast_df.loc[blast_df["sstart"]>blast_df["send"], "sorientation"] = "-"
        temp_df1 = blast_df.copy()
        blast_df.loc[blast_df["sorientation"]=="-", "sstart"] = blast_df["send"]
        blast_df.loc[blast_df["sorientation"]=="-", "send"] = temp_df1["sstart"]
    
        #filter self-alignment
        if self == True:
            blast_df = blast_df[(blast_df["qstart"]!=blast_df["sstart"])&(blast_df["qend"]!=blast_df["send"])]
            blast_df = blast_df[(blast_df["qstart"]!=blast_df["send"])&(blast_df["qend"]!=blast_df["sstart"])]
            blast_df = blast_df[blast_df["qstart"]<blast_df["sstart"]]
        
        #blast_df = blast_df.groupby(["qstart", "qend", "sstart", "send"]).first().reset_index()
        
        blast_df["q_mid"] = blast_df["qend"]-blast_df["qstart"]
        blast_df["q_mid"] = blast_df["q_mid"]/2+blast_df["qstart"]
        
        blast_df["s_mid"] = blast_df["send"]-blast_df["sstart"]
        blast_df["s_mid"] = blast_df["s_mid"]/2+blast_df["sstart"]
        
        blast_df["q_mid"] = blast_df["q_mid"].astype(int)
        blast_df["s_mid"] = blast_df["s_mid"].astype(int)
    return(blast_df)

In [ ]:
def calc_overlaps_wpartners(df, by):

    if by == "q":
        start_col = "qstart"
        end_col = "qend"
        start_col_2 = "sstart"
        end_col_2 = "send"
    elif by == "s":
        start_col = "sstart"
        end_col = "send"
        start_col_2 = "qstart"
        end_col_2 = "qend"
    elif by == "gtf":
        start_col = "Start"
        end_col = "End"    
    elif by =="LeftRight":
        start_col = "Left"
        end_col = "Right"
        
    df = df.sort_values(by=start_col).reset_index(drop=True)
    
    overlapping1 = []
    overlapping_coverage1 = []
    partner_coord_groups = []
    
    i = 0
    
    # Iterate through the sorted intervals
    while i < len(df):
        start = df.iloc[i][start_col]
        end = df.iloc[i][end_col]
        s2 = df.iloc[i][start_col_2]
        e2 = df.iloc[i][end_col_2]
        
        partner_coords = []
        partner_coords.append([s2,e2])
        
        overlap_group = [i]
        
        i += 1
        
        # Find overlapping intervals
        while i < len(df) and df.iloc[i][start_col] <= end:
            end = max(end, df.iloc[i][end_col])
            s2 = df.iloc[i][start_col_2]
            e2 = df.iloc[i][end_col_2]
            partner_coords.append([s2,e2])
            overlap_group.append(i)
            i += 1

        partner_coord_groups.append(partner_coords)
        overlapping1.append(overlap_group)
        overlapping_coverage1.append([start, end])

    # Create the 'overlap_df1' DataFrame
    overlap_df1 = pd.DataFrame({
        'Transcripts': overlapping1,
        'Coords': overlapping_coverage1,
        'TrNum': range(len(overlapping1)),
        'partner_coords': partner_coord_groups
    })
    overlap_df1["Left"] = [item[0] for item in overlap_df1["Coords"].values]
    overlap_df1["Right"] = [item[1] for item in overlap_df1["Coords"].values]
    
    return overlap_df1, overlapping_coverage1

In [ ]:
def calc_overlaps(df, by):

    if by == "q":
        start_col = "qstart"
        end_col = "qend"
    elif by == "s":
        start_col = "sstart"
        end_col = "send"
    elif by == "gtf":
        start_col = "Start"
        end_col = "End"    
    elif by =="LeftRight":
        start_col = "Left"
        end_col = "Right"
    else:
        start_col = by[0]
        end_col = by[1]
        
    df = df.sort_values(by=start_col).reset_index(drop=True)
    
    overlapping1 = []
    overlapping_coverage1 = []
    group_counter = 0
    i = 0
    
    # Iterate through the sorted intervals
    while i < len(df):
        start = df.iloc[i][start_col]
        end = df.iloc[i][end_col]
        overlap_group = [i]
        
        i += 1
        
        # Find overlapping intervals
        while i < len(df) and df.iloc[i][start_col] <= end:
            end = max(end, df.iloc[i][end_col])
            overlap_group.append(i)
            i += 1
        
        overlapping1.append(overlap_group)
        overlapping_coverage1.append([start, end])

    # Create the 'overlap_df1' DataFrame
    overlap_df1 = pd.DataFrame({
        'Transcripts': overlapping1,
        'Coords': overlapping_coverage1,
        'TrNum': range(len(overlapping1))
    })
    overlap_df1["Left"] = [item[0] for item in overlap_df1["Coords"].values]
    overlap_df1["Right"] = [item[1] for item in overlap_df1["Coords"].values]
    
    return overlap_df1, overlapping_coverage1

In [ ]:
def overlaps_to_plot(overlap_df1, overlapping_coverage1, seq, yoffset=1, xoffset=0):
    empty_regions = []
    for i in range(1, len(overlapping_coverage1)):
        if overlapping_coverage1[i - 1][1] + 1<overlapping_coverage1[i][0] - 1:
            empty_regions.append([overlapping_coverage1[i - 1][1] + 1, overlapping_coverage1[i][0] - 1])

        # [11,12][13,14]
        # 13,12
        # [11,12][14,15]
        # 13,13
    #empty_regions.append([0, overlapping_coverage1[0][0] - 1])
    if overlapping_coverage1[0][0] == 0:
        empty_regions.append([-100, overlapping_coverage1[0][0] - 1])
    else:
        empty_regions.append([0, overlapping_coverage1[0][0] - 1])
    if isinstance(seq,str):
        if overlapping_coverage1[-1][1] == len(seq):
            empty_regions.append([overlapping_coverage1[-1][1] + 1, len(seq)+100])
        else:
            empty_regions.append([overlapping_coverage1[-1][1] + 1, len(seq)+100])
    elif isinstance(seq,int):
        if overlapping_coverage1[-1][1] == seq:
            empty_regions.append([overlapping_coverage1[-1][1] + 1, seq+100])
        else:
            empty_regions.append([overlapping_coverage1[-1][1] + 1, seq+100])
    
    dfdf1 = pd.DataFrame(empty_regions, columns=['Left', 'Right'])
    dfdf1['Coverage'] = 0
    dfdf3 = pd.concat([dfdf1["Left"], dfdf1["Right"]], axis = 0)
    dfdf3 = dfdf3.to_frame(name = "Coords")
    dfdf3["Coverage"] = 0
    dfdf4 = pd.concat([overlap_df1["Left"], overlap_df1["Right"]], axis = 0)  
    dfdf4 = dfdf4.to_frame(name = "Coords")
    dfdf4["Coverage"] = yoffset
    tr_cov_df = pd.concat([dfdf3, dfdf4], axis = 0)
    tr_cov_df = tr_cov_df.sort_values(by="Coords")
    tr_cov_df["Coords"] = tr_cov_df["Coords"]+xoffset
    return tr_cov_df

In [ ]:
def gtf_to_plot(gtf_df, seq_len):
    gtf_df.sort_values(by='Start', inplace=True)
    
    # Initialize variables
    overlapping1 = []
    overlapping_coverage1 = []
    group_counter = 0
    i = 0
    
    # Iterate through the sorted intervals
    while i < len(gtf_df):
        start = gtf_df.iloc[i]['Start']
        end = gtf_df.iloc[i]['End']
        overlap_group = [i]
        
        i += 1
        
        # Find overlapping intervals
        while i < len(gtf_df) and gtf_df.iloc[i]['Start'] <= end:
            end = max(end, gtf_df.iloc[i]['End'])
            overlap_group.append(i)
            i += 1
        
        overlapping1.append(overlap_group)
        overlapping_coverage1.append([start, end])
    
    # Create the 'overlap_df1' DataFrame
    overlap_df1 = pd.DataFrame({
        'Transcripts': overlapping1,
        'Coords': overlapping_coverage1,
        'TrNum': range(len(overlapping1))
    })
    
    # Create the 'empty_regions' DataFrame
    empty_regions = []
    for i in range(1, len(overlapping_coverage1)):
        empty_regions.append([overlapping_coverage1[i - 1][1] + 1, overlapping_coverage1[i][0] - 1])
    
    empty_regions.append([0, overlapping_coverage1[0][0] - 1])
    empty_regions.append([overlapping_coverage1[-1][1] + 1, seq_len])
    
    dfdf1 = pd.DataFrame(empty_regions, columns=['Left', 'Right'])
    dfdf1['Coverage'] = 0
    dfdf3 = pd.concat([dfdf1["Left"], dfdf1["Right"]], axis = 0)
    dfdf3 = dfdf3.to_frame(name = "Coords")
    dfdf3["Coverage"] = 0
    overlap_df1["Left"] = [item[0] for item in overlap_df1["Coords"].values]
    overlap_df1["Right"] = [item[1] for item in overlap_df1["Coords"].values]
    dfdf4 = pd.concat([overlap_df1["Left"], overlap_df1["Right"]], axis = 0)  
    dfdf4 = dfdf4.to_frame(name = "Coords")
    dfdf4["Coverage"] = 1
    tr_cov_df_ = pd.concat([dfdf3, dfdf4], axis = 0)
    
    print("Done")
    return tr_cov_df_

In [ ]:
def parse_fasta(path, rev_comp=False, rm_barcode=False, pdf=True):
    ids = []
    seqs = []
    seq_df=[]
    if pdf:
        seq_df = pd.DataFrame()
        seq_list = []
        record_list = []
        
        for record in SeqIO.parse(path, "fasta"):
            record_list.append(record.id)
            seq_list.append(str(record.seq))
        
        seq_df["id"] = record_list
        seq_df["seq"] = seq_list
        
        if rm_barcode:
            seq_df["seq"] = seq_df["seq"].apply(lambda x: x[rm_barcode:-rm_barcode])
            
        if rev_comp == True:
            seq_df["seq_rev"] = seq_df["seq"].apply(lambda x: x[::-1])
            seq_df["seq_rev_comp"] = seq_df["seq"].apply(lambda x: str(Seq(x).reverse_complement()))
            seq_df["seq_comp"] = seq_df["seq"].apply(lambda x: str(Seq(x).complement()))
    
    else:
        file = open(path)
        i=1
        seqs = []
        ids = []
        if rm_barcode:
            for line in file:
                if i%2 == 0:
                    seqs.append(line[rm_barcode:-rm_barcode-1])
                    i+=1
                else:
                    ids.append(line[1:-1])
                    i+=1
        else:
            for line in file:
                if i%2 == 0:
                    seqs.append(line[:-1])
                    i+=1
                else:
                    ids.append(line[1:-1])
                    i+=1
                
    return seq_df, ids, seqs

In [ ]:
def write_fasta_from_fasta_df(seq_df, seq_col_name, id_col_name, path, rc=False):
    write_file = open(path, "w")
    for index, row in seq_df.iterrows():
        write_file.write(">" + seq_df[id_col_name][index] + "\n" + seq_df[seq_col_name][index] + "\n")
        if rc==True:
            write_file.write(">" + seq_df[id_col_name][index] + "_comp" + "\n" + seq_df["seq_comp"][index] + "\n")
            write_file.write(">" + seq_df[id_col_name][index] + "_rev" + "\n" + seq_df["seq_rev"][index] + "\n")
            write_file.write(">" + seq_df[id_col_name][index] + "_rev_comp"+ "\n" + seq_df["seq_rev_comp"][index] + "\n")
    write_file.close()

In [ ]:
def write_fasta_from_seq_list(seq_list, seq_names_list, path):
    write_file = open(path, "w")
    for i in range(len(seq_list)):
        write_file.write(">" + seq_names_list[i] + "\n" + seq_list[i] + "\n")
    write_file.close()

In [ ]:
def write_fasta_from_MREdf(MREdf, seq, save_path):
    write_file = open(save_path, "w")
    for index, row in MREdf.iterrows():
        temp_seq = seq[MREdf["MRE_zone_start"][index]:MREdf["MRE_zone_end"][index]+7]
        write_file.write(">" + str(MREdf["MRE_zone_start"][index]) + "-" + str(MREdf["MRE_zone_end"][index]+7) + "\n" + temp_seq + "\n")
    write_file.close()

In [ ]:
def write_fasta_from_overlap_df1(overlap_df1, seq, save_path):
    write_file = open(save_path, "w")
    for index, row in overlap_df1.iterrows():
        temp_seq = seq[overlap_df1["Left"][index]:overlap_df1["Right"][index]]
        write_file.write(">" + str(overlap_df1["Left"][index]) + "-" + str(overlap_df1["Right"][index]) + "\n" + temp_seq + "\n")
    write_file.close()

In [ ]:
def uniques_only(data, n=1):
    from collections import Counter
    histogram = Counter(data)
    uniques = [d for d in data if histogram[d] <= n]
    return uniques

In [ ]:
def uniques_only_list(data):
    from collections import OrderedDict
    list(OrderedDict.fromkeys(t))
    

In [ ]:
def write_list(list, save_path):
    write_list = open(save_path, "w")
    write_list.write("\n".join(list))
    write_list.write(("\n"))
    write_list.close()

In [ ]:
def read_list(path):
    with open(path)as f:
        out=f.read()
    return out

In [1704]:
def find_ORFs(seq, stops = ["TAA","TGA","TAG"], len_filter=50, to_translate=True):
    '''
    out: 0 - ORF_coords
         1 - ORF_seqs
         2 - ORF_stops
         3 - ORF_lens
         4 - ORF_prots
         5 - longest_ORF_coords
         6 - num_ORFs_found
    '''    
    ORF_coords=[]
    for j in range(0,3):
        leftover = (len(seq)-j)%3
        for i in range(0+j, len(seq)-leftover, 3):
            codon = seq[i:i+3]
            if codon == "ATG":
                start = i+1
                for z in range(i, len(seq)-leftover, 3):
                    codon = seq[z:z+3]
                    if codon in stops:
                        stop = z+3
                        ORF_coords.append([start,stop])
                        break
    ORF_lens = [x[1]-(x[0]-1) for x in ORF_coords]
    stops = [seq[x[1]-3:x[1]] for x in ORF_coords]
    seqs = [seq[x[0]-1:x[1]] for x in ORF_coords]

  
    seqs_above_filter_enum = [x for x in enumerate(ORF_lens) if x[1] > len_filter]
    longest_ORF_idx = [x[0] for x in seqs_above_filter_enum if x[1]==max(ORF_lens)]
    longest_ORF = ORF_coords[longest_ORF_idx[0]]
    index = [x[0] for x in seqs_above_filter_enum]
    ORFs_above_filter = [ORF_coords[i] for i in index]
    stops_above_filter = [stops[i] for i in index]
    lens_above_filter = [ORF_lens[i] for i in index]
    seqs_above_filter = [seqs[i] for i in index]
    num_ORFs_found = len(lens_above_filter)

    if to_translate==True:
        prots = [translate(x) for x in seqs_above_filter]
    else:
        prots = None
    out = [ORFs_above_filter, seqs_above_filter, stops_above_filter, lens_above_filter, prots, longest_ORF, num_ORFs_found]
    return out

In [18]:
def find_fltc_ORFs(seq, min_len=50, starts=r"ATG", stops_list=["TAA","TGA","TAG"], to_translate=True, all_ORFs=False):
    import pandas as pd
    
    ORF_df = []
    longest_ORF_coords = []
    
    i=1
    # num_stops = len(stops_list)
    # for stop in stops_list:
    #     stops_list += stop
    #     i+=1
    #     if i == num_stops:
    #         break
    #     else:
    #         stops += "|"
    
    stops=""
    for s in stops_list:
        stops+=s
        stops+="|"
    stops=stops[:-1]
    stops=fr"{stops}"

    # stops=r"TAA|TAG|TGA"
    pattern = fr'(?={starts}((?:TGA|TAG|TTT|TTA|TTG|TTC|TCA|TCC|TCT|TCG|TGG|TGT|TGC|TAT|TAC|AAA|AAG|AAC|AAT|AGA|AGG|AGT|AGC|ACA|ACC|ACG|ACT|ATA|ATT|ATC|GGG|GGC|GGT|GGA|GCG|GCC|GCA|GCT|GTA|GTT|GTG|GTC|GAT|GAA|GAG|GAC|CCC|CCA|CCG|CCT|CAC|CAA|CAT|CAG|CTG|CTC|CTT|CTA|CGG|CGC|CGA|CGT)+?)({stops}))'
    #ORFs = re.findall(fr'(?=({starts}(?:...)*?)(TAA))', seq)
    ORFs = re.findall(pattern, seq)
    ORFs_iter = re.finditer(pattern,seq)
    ORF_list = []
    ORF_len = []
    ORF_span = []
    ORF_stops = []
    ORF_start = []
    x=False
    j=-1
    ORF_idx = []

    if len(ORFs) > 0:
        for i in ORFs:
            j+=1
            if len(i[0])>min_len and len(i[0])%3==0:
                ORF_list.append(i[0]) 
                ORF_stops.append(i[1])
                ORF_len.append(len(i[0]))
                ORF_idx.append(j)
                x=True
    
        z=-1
        j=0
        if len(ORF_idx) > 0:
            for i in ORFs_iter:
                z+=1
                if z == ORF_idx[j]:
                    if j<len(ORF_idx)-1:
                        j+=1
                    if x == True:
                        span = list(i.span())
                        start = span[0]+4
                        ORF_start += [start]
    
            ORF_end = []
            for i in range(len(ORF_start)):
                ORF_end.append(ORF_start[i]+ORF_len[i]-1)
        
            ORF_df = pd.DataFrame([ORF_list, ORF_stops, ORF_len, ORF_start, ORF_end])
            ORF_df = ORF_df.transpose()
            ORF_df.columns = ["seq", "stop", "seq_len", "start_coord", "last_coord"]
            ORF_df["seq"] = ORF_df.apply(lambda x: "ATG" + x.seq + x.stop, axis=1)
            ORF_df["start_coord"] = ORF_df["start_coord"]-3
            ORF_df["last_coord"] = ORF_df["last_coord"]+3
            ORF_df["seq_len"] = ORF_df["seq_len"]+6
            
            longest_ORF_idx = ORF_df["seq_len"].idxmax()
            longest_ORF_coords = [ORF_df["start_coord"][longest_ORF_idx],ORF_df["last_coord"][longest_ORF_idx]]
            if all_ORFs:
                ORF_df["all_ORF_coords"] = ORF_df.apply(lambda x: [x.start_coord, x.last_coord],axis=1)
                all_ORFs = ORF_df["all_ORF_coords"].tolist()
            if to_translate == True:
                ORF_df["prot"] = ORF_df["seq"].apply(lambda x: translate(x, stops=stops))
        else:
            print("Discovered ORFs are too short")
    else:
        print("No ORFs found")
        
    num_ORFs_found = len(ORF_df) 
    return ORF_df, longest_ORF_coords, num_ORFs_found, all_ORFs

In [ ]:
def find_ORFs_old(seq, min_len=50, starts="ATG", stops="TAA|TAG|TGA"):
    ORFs = re.findall(fr'{starts}(.*?)({stops})', seq)
    ORF_list = []
    for i in ORFs:
        if len(i[0])>min_len and len(i[0])%3==0:
            ORF = starts + i[0] + i[1]
            ORF_list += [ORF]
    return ORF_list

In [ ]:
#for every item in the list produces a lists of item indeces
def list_duplicates(int_list):
    from collections import defaultdict
    tally = defaultdict(list)
    for i, item in enumerate(int_list):
        tally[item].append(i)
    return ((key, locs) for key, locs in tally.items()
           if len(locs)>1)

# for dup in sorted(list_duplicates(all_repeats)):
#     print(dup)
# to view result

In [ ]:
def fasta_subseqs(fasta_seq, left_right_df):
    left_right_df["Seq"] = left_right_df.apply(lambda x: [fasta_seq[x.Left+1, x.Right]], axis=1)

In [ ]:
def calc_nucl_freq(seq_list):
    #ACGT x sequence length
    #list of sequences as input
    from collections import Counter
    lines = [list(line[:]) for line in seq_list]
    #nucl_frequencies = [[0]*len(lines[0]) for _ in range(4)]
    nucl_frequencies = np.zeros([4, len(lines[0])])
    lines = np.array(lines)
    for i in range(lines.shape[1]):
        counter = Counter(lines[:,i])
        nuc_types = len(counter)
        counter["A"] += 0
        counter["C"] += 0
        counter["G"] += 0
        counter["T"] += 0
        j = 0
        for key in sorted(counter.keys()):
            nucl_frequencies[j][i] = counter[key]/nuc_types
            j+=1
    return nucl_frequencies

In [4]:
def calc_aa_freq(seq_list):
    #ACDEFGHIKLMNPQRSTVWY x sequence length
    #list of sequences as input
    from collections import Counter
    lines = [list(line[:]) for line in seq_list]
    #aa_frequencies = [[0]*len(lines[0]) for _ in range(4)]
    aa_frequencies = np.zeros([20, len(lines[0])])
    lines = np.array(lines)
    for i in range(lines.shape[1]):
        counter = Counter(lines[:,i])
        aa_types = len(counter)
        counter["R"] += 0
        counter["K"] += 0
        counter["D"] += 0
        counter["E"] += 0
        counter["S"] += 0
        counter["T"] += 0
        counter["N"] += 0
        counter["Q"] += 0
        counter["A"] += 0
        counter["V"] += 0
        counter["L"] += 0
        counter["I"] += 0
        counter["M"] += 0
        counter["C"] += 0
        counter["F"] += 0
        counter["Y"] += 0
        counter["W"] += 0
        counter["H"] += 0
        counter["G"] += 0
        counter["P"] += 0
        j = 0
        for key in sorted(counter.keys()):
            aa_frequencies[j][i] = counter[key]/aa_types
            j+=1
    return aa_frequencies

In [3]:
def count_aas(seq):
    #ACDEFGHIKLMNPQRSTVWY x sequence length
    #list of sequences as input
    from collections import Counter
    seq = list(seq)
    aa_counts = Counter(seq)
    return aa_counts

In [2]:
def DNA_onehot(seq):
    encoded = []
    embeddings = {"C":[1.,0.,0.,0.], "G":[0.,1.,0.,0.], "A":[0.,0.,1.,0.], "T":[0.,0.,0.,1.]}
    for i in seq:
        encoded.append(embeddings[i] if i in embeddings.keys() else [0.,0.,0.,0.])
    return np.array(encoded)

In [7]:
def count_kmers(seq, klen, mode="DNA"):
    import itertools as it
    kmer_dict = {}
    if mode == "DNA":
        DNA = "ACTGn"
    elif mode == "protein":
        DNA = "ACDEFGHIKLMNPQRSTVWY*"
    for output in it.product(DNA,repeat=klen):
        kmer_dict["".join(output)]=0
    num_kmers = len(seq) - klen +1
    for i in range(num_kmers):
        kmer=seq[i:i+klen]
        if "N" not in kmer:
            kmer_dict[kmer] +=1
    return kmer_dict

In [32]:
#needs multiple sequences?
def find_overrepresented_aas(seq,pseudo_p=0.05):
    
    import pandas as pd
    seq_len = len(seq)
    letter_counts = count_kmers(seq,1,mode="protein")
    letter_probs = dict()
    for key in letter_counts.keys():
        letter_probs[key] = letter_counts[key]/seq_len
    letter_probs_df = pd.DataFrame()
    letter_probs_df["letter"] = letter_counts.keys()
    letter_probs_df["pseudo_p"] = pseudo_p
    letter_probs_df["pseudo_p_count"] = pseudo_p*len(seq)
    letter_probs_df["seq_p"] = letter_probs.values()
    letter_probs_df["seq_p_count"] = letter_counts.values()
    std_dev = 2+2
    std_err = std_dev/math.sqrt(len(seq))
    letter_probs_df["diff"] = letter_probs_df["seq_p"]-letter_probs_df["pseudo_p"]
    letter_probs_df["t_stat"] = letter_probs_df["diff"]/std_err
    
    overrepresented_letters = []
    return letter_probs_df, overrepresented_letters

In [29]:
#compare two populations of data e.g. two sets of sequences
def mann_whitney(data1, data2, p=0.05):
    import pandas as pd
    data_df1 = pd.DataFrame()
    data_df2 = pd.DataFrame()
    data_df = pd.DataFrame()
    data_df1["data"] = data1
    data_df2["data"] = data2
    data_df1["label"] = 1
    data_df2["label"] = 2 
    data_df = pd.concat([data_df1, data_df2])
    data_df = data_df.sort_values(by="data").reset_index(drop=True)
    data_df["index"] = data_df.index
    data1_R = data_df[data_df["label"]==1]["index"].sum()
    data2_R = data_df[data_df["label"]==2]["index"].sum()
    n1 = data_df1.shape[0]
    n2 = data_df2.shape[0]
    U1 = n1*n2+(n1*(n1+1))/2 - data1_R
    U2 = n2*n1+(n2*(n2+1))/2 - data2_R
    U = min(U1,U2)
    print("n1:", n1, "  |  ", "n2:", n2)
    return(U)

In [ ]:
def t_test(data1, data2, p=0.05, sample_variances="equal"):
    import math
    n1 = len(data1)
    n2 = len(data2)
    deg_freedom = n1+n2-2
    sum1 = sum(data1)
    sum2 = sum(data2)
    mean1 = sum1/n1
    mean2 = sum2/n2
    mean_diff = abs(mean1-mean2)
    
    sum_diff1 = 0
    for i in data1:
        sum_diff1 += (i-mean1)**2
    st_dev1 = math.sqrt(sum_diff1/n1)
    
    sum_diff2 = 0
    for j in data2:
        sum_diff2 += (i-mean2)**2
    st_dev2 = math.sqrt(sum_diff2/n2)
    if sample_variances == "equal":
        pvar = (((n1-1)*st_dev1**2)+((n2-1)*st_dev2**2))/(n1+n2-2)
    
        t = mean_diff/(math.sqrt(pvar)*math.sqrt(1/n1+1/n2))
    elif sample_variances == "unequal":
        t = mean_diff/math.sqrt(st_dev1**2/n1+st_dev2**2/n2)
    return t, deg_freedom

In [1]:
def extract_nested_values(it):
    if isinstance(it, list):
        for sub_it in it:
            yield from extract_nested_values(sub_it)
    elif isinstance(it, dict):
        for value in it.values():
            yield from extract_nested_values(value)
    else:
        yield it

In [ ]:
def PCA_feature_importance(df):
    pca = PCA()
    x_new = pca.fit_transform(df.values)
    
    def myplot(score, coeff, labels=None):
        xs = score[:,0]
        ys = score[:,1]
        n=coeff.shape[0]
        scalex = 1/(xs.max() - xs.min())
        scaley = 1/(ys.max() - ys.min())
        plt.scatter(xs*scalex,ys*scaley,c="y")
        for i in range(n):
            plt.arrow(0, 0, coeff[i,0], coeff[i,1], color= "r", alpha=0.5)
            if labels is None:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] *1.15, "Var"+str(i+1), color="g", ha = "center", va= "center")
                Z=0
            else:
                plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color="g", ha="center", va="center")
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(1))
    plt.grid()
    
    myplot(x_new[:,0:2], np.transpose(pca.components_[0:2, :]))
    plt.show()
    return x_new, pca

In [ ]:
def convert_tr_ids(tr_ids,to="tissue"):
    tr_id_conversion = pd.read_csv("/home/maxim/PhD/Project/Data/Long_read_RNAseq/lrRNAseq_barcode_deconvolution/final/blast_final_ids", header=None)
    tr_id_conversion.columns=["id"]
    tr_id_conversion["id"]=tr_id_conversion["id"].apply(lambda x: x[1:])
    tr_id_conversion["id_c"]=tr_id_conversion["id"].apply(lambda x: re.match(r"(.*?)(ccs)",x)[0])

    if to=="tissue":
        tr_converted = tr_id_conversion[tr_id_conversion["id_c"].isin(tr_ids)]["id"].tolist()
        tr_converted_dict = dict(tr_id_conversion[tr_id_conversion["id_c"].isin(tr_ids)].values)
        c_dict = {v: k for k,v in tr_converted_dict.items()}
    elif to=="plain":
        tr_converted = tr_id_conversion[tr_id_conversion["id"].isin(tr_ids)]["id_c"].tolist()
        tr_converted_dict = dict(tr_id_conversion[tr_id_conversion["id"].isin(tr_ids)].values)
        c_dict = {v: k for k,v in tr_converted_dict.items()}
    return tr_converted,c_dict

In [ ]:
def calc_GC(seq):
    G = seq.count("G")
    C = seq.count("C")
    seq_len = len(seq)
    GC = 100/seq_len*(G+C)
    return GC

In [ ]:
#process seq subset after removing barcodes
def fix_barcode_stat_df(stat_df, unique_id, db="Lr13H12",blast=False):
    #forgot to remove barcodes on a small subset of stat df and blast? quickly re-do both here
    stat_df_cut = stat_df.copy()
    stat_df_cut["seq"] = stat_df_cut["seq"].apply(lambda x: x[39:-39])
    if blast:
        write_fasta_from_fasta_df(stat_df_cut,"seq","id",f"lrRNAseq_barcode_deconvolution/barcode_fixing/{unique_id}")
        os.system(f"blastn -query /home/maxim/PhD/Project/Data/Long_read_RNAseq/lrRNAseq_barcode_deconvolution/barcode_fixing/{unique_id} -db /home/maxim/PhD/Project/Data/Sequencing/shinobus_BACs/BLAST_full_BAC/{db}_db -max_hsps 10000 -task blastn -evalue 1e-2 -num_threads 4 -out /home/maxim/PhD/Project/Data/Long_read_RNAseq/lrRNAseq_barcode_deconvolution/barcode_fixing/{unique_id}_blast.out -outfmt '6 qseqid sseqid slen qlen qstart qend sstart send length mismatch gapopen pident evalue bitscore' -word_size 4")
        blast_df = read_blast(f"/home/maxim/PhD/Project/Data/Long_read_RNAseq/lrRNAseq_barcode_deconvolution/barcode_fixing/{unique_id}_blast.out")
    else:
        blast_df=[]
    new_stat_df = find_tr_ORF_stats(stat_df_cut["seq"].tolist(), stat_df_cut["id"].tolist(), include_seq=True, tissue=1)
    return blast_df, new_stat_df

In [148]:
def local_substring_occurrences(seq,subseq,window,skip=0):
    #rolling window counter of substrings in local area
    from operator import add
    seq_len=len(seq)
    window_range=seq_len-2*window
    counts=[0]*seq_len
    if isinstance(subseq,list):
        for ss in subseq:
            c=[]
            for i in range(window):
                sample_seq=seq[:i]
                count = substring_occurrences(sample_seq,ss)
                c.append(count)
    
            i=0
            while i<window_range:
                if skip:
                    if i%skip==0:
                        i+=1
                sample_seq = seq[i:i+window]
                count = substring_occurrences(sample_seq,ss)
                c.append(count)
                i+=1
        
            for i in range(window,0,-1):
                sample_seq=seq[-i:]
                count = substring_occurrences(sample_seq,ss)
                c.append(count)
            counts=list(map(add,counts,c))
    else:
        for i in range(window):
            sample_seq=seq[:i]
            count = substring_occurrences(sample_seq,subseq)
            counts.append(count)
    
        i=0
        while i<window_range:
            if skip:
                if i%skip==0:
                    i+=1
            sample_seq = seq[i:i+window]
            count = substring_occurrences(sample_seq,subseq)
            counts.append(count)
            i+=1
    
        for i in range(window,0,-1):
            sample_seq=seq[-i:]
            count = substring_occurrences(sample_seq,subseq)
            counts.append(count)
            
    return counts

In [ ]:
def ranges_over_depth_threshold(depth, coords, depth_threshold, df=False):
    
    depth_enum = [x for x in enumerate(depth) if x[1] > depth_threshold]
    depth_index = [i[0] for i in depth_enum]
    desired_depth = [i[1] for i in depth_enum]
    desired_coords = [coords[i] for i in depth_index]
    
    high_depth_ranges=[]
    for i in range(len(desired_coords)-1):
        if i == 1:
            start = desired_coords[i]
        if desired_coords[i+1] != desired_coords[i]+1:
            end = desired_coords[i]
            high_depth_ranges.append([start,end])
            start = desired_coords[i+1]
    end=desired_coords[i+1]
    high_depth_ranges.append([start,end])

    if df==True:
        high_depth_ranges = pd.DataFrame(high_depth_ranges)
        high_depth_ranges.columns=["Start","End"]
    
    return high_depth_ranges, desired_depth, desired_coords

In [ ]:
def extract_subseqs_by_coords(seq, coords=[], left=[], right=[]):
    subseq_list = []
    if len(coords)>0:
        for i in coords:
            subseq_list.append(seq[i[0]-1:i[1]])
            
        x=1+1
    elif len(left)>0:
        for i in range(len(left)):
            subseq_list.append(seq[left[i]-1:right[i]])
    return subseq_list

In [ ]:
def subseqs_by_depth_threshold(seq, depth, coords, depth_threshold):
    r, dd, dc = ranges_over_depth_threshold(depth, coords, depth_threshold)
    subseq_list = extract_subseqs_by_coords(seq, r)
    return subseq_list,r

In [11]:
def translate_all_pretty(seq, display_seq=True, stops=[]):
    if display_seq:
        print("seq",seq)
    if stops:
        t = translate(seq, frame=16, stops=stops)
    else:
        t = translate(seq, frame=16)
    seq_states = ["fwd", "rev", "comp", "rev_comp"]
    for i in range(len(t)):
        curr_seq_state = int((i)/3)
        print("frame:", i+1, "- ", seq_states[curr_seq_state])
        print(t[i])

In [ ]:
def display_high_depth_regions(seq, depth, coords, depth_threshold):
    s,r=subseqs_by_depth_threshold(seq, depth, coords, depth_threshold)
    for i in range(1,len(s)+1):
        print("seq", i, "-", s[i-1])
        translate_all_pretty(s[i-1], display_seq=False)
        print("\n =========================== \n")

In [661]:
def pseudorandom_gen(seq_len, CGAT_p=[25,25,25,25]):
    import numpy as np
    sum_p = sum(CGAT_p)
    seq_gen = list(np.random.randint(1,sum_p+1, seq_len))
    pseudoseq=""
    c = CGAT_p[0]
    g = CGAT_p[1]+c
    a = CGAT_p[2]+g
    t = CGAT_p[3]+a
    for x in seq_gen:
        if x<=c:
            pseudoseq = pseudoseq + "C"
        elif x>c and x <=g:
            pseudoseq = pseudoseq + "G"
        elif x>g and x <=a:
            pseudoseq = pseudoseq + "A"
        else:
            pseudoseq = pseudoseq + "T"
    return pseudoseq

In [4]:
def repetitiveness_index(seq, longest_seq_len=10000):
    k=4
    i=1
    while k < longest_seq_len:
        k=4
        k=k**i
        i+=1
        # print(i)
    kmer_count_range = []

    for j in range(i):
        kmer_count_range.append(4**j)
    # print(kmer_count_range)
    seq_len = len(seq)
    # print(seq_len)
    for z in range(1,i):
        if seq_len>=kmer_count_range[z-1] and seq_len<=kmer_count_range[z]:
            klen=z
            
    kmer_ceiling = 4**klen
    max_kmers = seq_len-klen+1
    kmer_dict = count_kmers1(seq, klen)
    kmer_num = len(kmer_dict.keys())
    kmer_uniqueness = 100/max_kmers*kmer_nm
    kmer_uniqueness_len = 100/max_kmers*kmer_num*seq_len
    kmer_uniqueness_klen = 100/max_kmers*kmer_num*(4**klen)
    unbound_uniqueness = 100/kmer_ceiling*kmer_num
    unbound_uniqueness_len = 100/kmer_ceiling*kmer_num*seq_len
    unbound_uniqueness_klen = 100/kmer_ceiling*kmer_num*(4**klen)
    return kmer_uniqueness, kmer_uniqueness_len, kmer_uniqueness_klen, unbound_uniqueness, unbound_uniqueness_len, unbound_uniqueness_klen, kmer_ceiling, max_kmers, kmer_num, klen

In [5]:
def repetitiveness_index_full_scale(seq, klen):
    seq_len = len(seq)
    
    kmer_ceiling = 4**klen
    max_kmers = seq_len-klen+1
    kmer_dict = count_kmers1(seq, klen)
    kmer_num = len(kmer_dict.keys())
    kmer_uniqueness = 100/max_kmers*kmer_num
    kmer_uniqueness_len = 100/max_kmers*kmer_num*seq_len
    kmer_uniqueness_klen = 100/max_kmers*kmer_num*(4**klen)
    unbound_uniqueness = 100/kmer_ceiling*kmer_num
    unbound_uniqueness_len = 100/kmer_ceiling*kmer_num*seq_len
    unbound_uniqueness_klen = 100/kmer_ceiling*kmer_num*(4**klen)
    return kmer_uniqueness, kmer_uniqueness_len, kmer_uniqueness_klen, unbound_uniqueness, unbound_uniqueness_len, unbound_uniqueness_klen, kmer_ceiling, max_kmers, kmer_num, klen

In [ ]:
def repetitiveness_index_accumulative(seq, klen, previous=0, prev_len=0):
    seq_len = len(seq)
    
    max_kmers = (seq_len-klen+1)+prev_len
    kmer_dict = count_kmers1(seq, klen)
    if previous != 0:
        for k in previous.keys():
            if k in kmer_dict.keys():
                kmer_dict[k] += previous[k]
            else:
                kmer_dict[k] = previous[k]
    kmer_num = len(kmer_dict.keys())
    kmer_uniqueness = 100/max_kmers*kmer_num
    return kmer_uniqueness, max_kmers, kmer_num, kmer_dict

In [ ]:
def repetitiveness_index_ratio_corrected(seq, klen):
    seq_len = len(seq)
    
    kmer_ceiling = 4**klen
    max_kmers = seq_len-klen+1
    kmer_dict = count_kmers1(seq, klen)
    kmer_num = len(kmer_dict.keys())
    kmer_uniqueness = 100/max_kmers*kmer_num
    kmer_num_scaled = kmer_num/max_kmers*kmer_ceiling
    return kmer_num_scaled, kmer_uniqueness

In [1]:
def segment_fasta_blast(blast_path, fasta_path="", span=300000, extract=False, raw_path="", work_dir=""):
    if extract:
        blast = read_blast(blast_path)
        blast["soffset"] = 0
        blast.loc[(blast["slen"]>span)&(blast["sstart"]>span/2), "soffset"] = blast["sstart"]-(span/2)
        blast.loc[(blast["slen"]>span)&(blast["sstart"]>span/2), "send"] = blast["send"]-blast["soffset"]
        blast.loc[(blast["slen"]>span)&(blast["sstart"]>span/2), "sstart"] = span/2
        blast["slen_init"] = blast["slen"]
        blast.loc[blast["slen"]>span, "slen"] = span
        os.chdir(work_dir)
        ids_list = blast.sort_values("slen", ascending=False).drop_duplicates("sseqid")["sseqid"].astype(str).tolist()
        write_list(ids_list, "ids_ordered.txt")
        blast_fasta = blast.sort_values("sseqid").drop_duplicates("sseqid").reset_index(drop=True)
        raw_fasta_path = work_dir+"/raw_seqs.fasta"
        os.system(f"seqtk subseq {raw_path} {"ids_ordered.txt"} > {raw_fasta_path}")
        fasta_df = parse_fasta(raw_fasta_path)[0]
        fasta_df["soffset"] = blast_fasta["soffset"]
        fasta_df["slen_init"] = blast_fasta["slen_init"]
        fasta_df["seq_red"] = fasta_df["seq"]
        fasta_df.loc[fasta_df["soffset"]!=0,"seq_red"] = fasta_df.apply(lambda x: x.seq[int(x.soffset-span/2):int(x.soffset+span/2)], axis=1)
        fasta_df.loc[fasta_df["soffset"]==0,"seq_red"] = fasta_df.apply(lambda x: x.seq[0:span], axis=1)
        fasta_df = fasta_df.sort_values(by="slen_init", ascending=False)
        fasta_path = work_dir + "/red_seqs.fasta"
        write_fasta_from_fasta_df(fasta_df, "seq_red", "id", fasta_path)
        red_blast_path = work_dir + "/red_blast.out"
        blast = blast.sort_values(by="slen_init",ascending=False)
        blast.to_csv(red_blast_path, sep="\t", index=False)
        return fasta_df, blast, ids_list,blast_fasta

In [31]:
def sort_with_order(values, make_equal=False, make_sequential=False):
    '''
    Takes a list of values, sorts the values in ascending order,
    keeps track of the changes in original order of items in a list.
    Returns list of sorted values and a list of item order before sorting.
    '''
    enum = list(enumerate(values))
    order = [x[0] for x in enum]
    values = [x[1] for x in enum]
    old_values = [0]*len(values)
    ori_order = order.copy()

    #loops over values, if value list[i] is greater than value list[i+1], swaps their position and tracks original order
    while values != old_values:
        old_values = values.copy()
        for i in range(len(values)-1):
            if values[i]>values[i+1]:
                old = values[i]
                old_o = order[i]
                values[i] = values[i+1]
                order[i] = order[i+1]
                values[i+1]=old
                order[i+1]=old_o
                
    new_order = order.copy()
    idxs = []
    idxs_all = []
    uniq_values = []
    idxs_all_old = []
    
    for x in values:
        if x not in uniq_values:
            uniq_values.append(x)

    #loops over unique values, records index of each value occurence, sets order of each value to the minimal index 
    if make_equal:
        for i in uniq_values:
            idxs=[]
            for j in range(len(values)):
                if i==values[j]:
                    idxs.append(order[j])
            min_idx = min(idxs)
            dup_num = len(idxs)
            for idx in idxs:
                new_order[idx]=min_idx
                idxs_all.append(min_idx)
        order = idxs_all.copy()

    if make_sequential:
        for i in uniq_values:
            idxs=[]
            for j in range(len(values)):
                if i==values[j]:
                    idxs.append(order[j])
            min_idx = min(idxs)
            dup_num = len(idxs)
            for idx in idxs:
                new_order[idx]=min_idx
                idxs_all.append(min_idx)
        
        num_uniq = len(uniq_values)-1

        idxs_all_old = idxs_all.copy()
        
        existing=1
        
        while max(idxs_all)!=num_uniq:
            for i in range(len(idxs_all)):
                if i not in idxs_all and existing==1:
                    set_to = i
                    # print(set_to)
                    existing=0
                if existing==0:
                    # print(i)
                    for j in range(len(idxs_all)):
                        if idxs_all[j] == i:
                            idxs_all[j] = set_to
                            existing=1
                            # print(j)
            existing=1
            
        order = idxs_all.copy()
        
    return values, order
    # return values, order, new_order, idxs, idxs_all_old, idxs_all, uniq_values

In [31]:
def sort_with_order(values, make_equal=False, make_sequential=False):
    '''
    Takes a list of values, sorts the values in ascending order,
    keeps track of the changes in original order of items in a list.
    Returns list of sorted values and a list of item order before sorting.
    '''
    enum = list(enumerate(values))
    order = [x[0] for x in enum]
    values = [x[1] for x in enum]
    old_values = [0]*len(values)
    ori_order = order.copy()

    #loops over values, if value list[i] is greater than value list[i+1], swaps their position and tracks original order
    while values != old_values:
        old_values = values.copy()
        for i in range(len(values)-1):
            if values[i]>values[i+1]:
                old = values[i]
                old_o = order[i]
                values[i] = values[i+1]
                order[i] = order[i+1]
                values[i+1]=old
                order[i+1]=old_o
                
    new_order = order.copy()
    idxs = []
    idxs_all = []
    uniq_values = []
    idxs_all_old = []
    
    for x in values:
        if x not in uniq_values:
            uniq_values.append(x)

    #loops over unique values, records index of each value occurence, sets order of each value to the minimal index 
    if make_equal:
        for i in uniq_values:
            idxs=[]
            for j in range(len(values)):
                if i==values[j]:
                    idxs.append(order[j])
            min_idx = min(idxs)
            dup_num = len(idxs)
            for idx in idxs:
                new_order[idx]=min_idx
                idxs_all.append(min_idx)
        order = idxs_all.copy()

    if make_sequential:
        for i in uniq_values:
            idxs=[]
            for j in range(len(values)):
                if i==values[j]:
                    idxs.append(order[j])
            min_idx = min(idxs)
            dup_num = len(idxs)
            for idx in idxs:
                new_order[idx]=min_idx
                idxs_all.append(min_idx)
        
        num_uniq = len(uniq_values)-1

        idxs_all_old = idxs_all.copy()
        
        existing=1
        
        while max(idxs_all)!=num_uniq:
            for i in range(len(idxs_all)):
                if i not in idxs_all and existing==1:
                    set_to = i
                    # print(set_to)
                    existing=0
                if existing==0:
                    # print(i)
                    for j in range(len(idxs_all)):
                        if idxs_all[j] == i:
                            idxs_all[j] = set_to
                            existing=1
                            # print(j)
            existing=1
            
        order = idxs_all.copy()
        
    return values, order
    # return values, order, new_order, idxs, idxs_all_old, idxs_all, uniq_values

In [32]:
# values = [1,600,1,600]
# # values = [2,35, 5, 35,12,33,4]
# sv, order = sort_with_order(values)
# sort_with_order(values)

([1, 1, 600, 600], [0, 2, 1, 3])

In [33]:
# uniq_values = []

# for x in values:
#     if x not in uniq_values:
#         uniq_values.append(x)

# for i in uniq_values:
#     idxs=[]
#     for j in range(len(values)):
#         if i==values[j]:
#             idxs.append(order[j])
#     min_idx = min(idxs)
#     dup_num = len(idxs)
#     for idx in idxs:
#         new_order[idx]=min_idx
#         idxs_all.append(min_idx)
# order = idxs_all.copy()


# '''
# create list of [0] * len(order)
# populate indexes of list with the minimal index value of order items by index - similar to make_equal, but applied to original values order

# '''

SyntaxError: incomplete input (2664944077.py, line 8)

In [34]:
# sort_with_order(values, make_equal=1)

([1, 1, 600, 600], [0, 0, 1, 1])

In [35]:
# sv1, o1 = sort_with_order(values,make_sequential=1)
# sort_with_order(values,make_sequential=1)

([1, 1, 600, 600], [0, 0, 1, 1])

In [ ]:
# for i in range(len(sv)):
#     val = sv[i]
#     for j in range(len(sv)):
#         if val == sv1[j]:
            

In [11]:
def translate_order(order):
    '''
    #outputs sequential list of values, ascending in value every time an input list value changes
    '''
    new_order = []
    uniques = get_unique(order)
    for i in range(len(uniques)):
        for j in range(len(order)):
            if order[j]==uniques[i]:
                new_order.append(i)
                
    return new_order

In [13]:
def get_unique(values_list):
    
    uniq_values=[]
    for x in values_list:
        if x not in uniq_values:
            uniq_values.append(x)
            
    return uniq_values

In [ ]:
def space_elements(list_x, spacing=250):
    for i in range(len(list_x)-1):
        if list_x[i+1]-list_x[i] < spacing:
            list_x[i+1]=list_x[i]+spacing
    return list_x

In [ ]:
def syntenous_indexes(synteny_list):
    structure = []
    start=1
    for i in range(len(synteny_list)-1):
        if synteny_list[i+1]>= synteny_list[i] and start!=-1:
            start = -1
            structure.append(i)
        elif synteny_list[i+1]<synteny_list[i]:
            structure.append(i)
            start=1
    structure.append(len(synteny_list)-1)
    return structure